<a href="https://colab.research.google.com/github/erwinsot/Cuda/blob/main/Practica_3_paralela.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin
!wget --no-check-certificate \
    https://raw.githubusercontent.com/computationalcore/introduction-to-opencv/master/assets/haarcascade_frontalface_default.xml \
    -O haarcascade_frontalface_default.xml
!wget --no-check-certificate \
    https://raw.githubusercontent.com/computationalcore/introduction-to-opencv/master/utils/common.py \
    -O common.py

In [ ]:
!pip install pycuda # install cuda
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

In [3]:
%%writefile gaussian_blur.cu
__global__ void applyFilter(const unsigned char *input, unsigned char *output, const unsigned int corx, const unsigned int cory,const unsigned int width, const unsigned int height,const unsigned int width2, const unsigned int height2, const float *kernel, const unsigned int kernelWidth) {

   
    const unsigned int col2 = threadIdx.x  + blockIdx.x * blockDim.x;  
    const unsigned int row2 = threadIdx.y + blockIdx.y * blockDim.y;
    
    
    if(row2 >=cory && row2 <= height2+cory &&  col2 >=corx && col2 <= width2+corx) {
        const int half = kernelWidth / 2;        
        float blur = 0.0;
        for(int i = -half; i <= half; i++) {
            for(int j = -half; j <= half; j++) {

                const unsigned int y = max(0, min(height -1 , row2 + i));
                const unsigned int x = max(0, min(width -1, col2 + j));
                //printf("%d %d \n", y,x);

                const float w = kernel[(j + half) + (i + half) * kernelWidth];
                blur += w * input[x + y * width];
            }
        }
        output[col2 + row2 * width] = static_cast<unsigned char>(blur);
    }
    else
    { 
      output[col2 + row2 * width]= input[col2 + row2 * width];
    }   
   
}

Writing gaussian_blur.cu


In [71]:
%%writefile main.py
import numpy as np
import cv2
import os
import imutils
from PIL import Image
import pycuda.autoinit
import pycuda.driver as drv
import pycuda.compiler as compiler
import numpy as np
import math
import sys
import timeit
from PIL import Image
import cv2
import glob

personName = 'DetecFace'
personName2= 'BlurFace'
dataPath = '/content/images3'     #Cambia a la ruta donde hayas almacenado Data
dataPathTxt = '/content/images3'
personPath = dataPath + '/' + personName
personPath2 = dataPath + '/' + personName2

if not os.path.exists(personPath):
    print('Carpeta creada: ',personPath)
    os.makedirs(personPath)
if not os.path.exists(personPath2):
    print('Carpeta creada: ',personPath2)
    os.makedirs(personPath2)
#cap = cv2.VideoCapture(0,cv2.CAP_DSHOW)
cap = cv2.VideoCapture('ros.mp4')
faceClassif = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
count = 0


# ######################################## #
# generate gaussian kernel (size of N * N) #
# ######################################## #

sigma = 9  # standard deviation of the distribution
kernel_width = int(3 * sigma)
if kernel_width % 2 == 0:
    kernel_width = kernel_width - 1  # make sure kernel width only sth 3,5,7 etc

# create empty matrix for the gaussian kernel #
kernel_matrix = np.empty((kernel_width, kernel_width), np.float32)
kernel_half_width = kernel_width // 2
for i in range(-kernel_half_width, kernel_half_width + 1):
    for j in range(-kernel_half_width, kernel_half_width + 1):
        kernel_matrix[i + kernel_half_width][j + kernel_half_width] = (
                np.exp(-(i ** 2 + j ** 2) / (2 * sigma ** 2))
                / (2 * np.pi * sigma ** 2)
        )
gaussian_kernel = kernel_matrix / kernel_matrix.sum()

# start time
time_started = timeit.default_timer()

while True:    
    ret, frame = cap.read()    
    if ret == False: break
    frame =  imutils.resize(frame, width=640)    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    auxFrame = frame.copy()
    faces = faceClassif.detectMultiScale(gray,1.3,5)
    result = faces[0].reshape(1,-1).flatten().tolist()
    height2=result[2]
    width2=result[3]
    corx=result[0]
    cory=result[1]    
    for (x,y,w,h) in faces:
        cv2.rectangle(frame, (x,y),(x+w,y+h),(0,255,0),2)
        rostro = auxFrame[y:y+h,x:x+w]
        #rostro=cv2.GaussianBlur(rostro,(23,23),30)
        frame[y:y+rostro.shape[0], x:x+rostro.shape[1]] = rostro
        
        #rostro = cv2.resize(rostro,(150,150),interpolation=cv2.INTER_CUBIC)
        
        cv2.imwrite(personPath + '/rotro_{}.jpg'.format(count),frame) 
        try:
            img = Image.open(personPath + '/rotro_{}.jpg'.format(count))
            input_array = np.array(img)
            red_channel = input_array[:, :, 0].copy()
            green_channel = input_array[:, :, 1].copy()
            blue_channel = input_array[:, :, 2].copy()
          
        except FileNotFoundError:
            sys.exit("Cannot load image file")

        # #################################################################### #
        # calculate the CUDA threats/blocks/gird base on width/height of image
        # #################################################################### #
        height, width = input_array.shape[:2]
        dim_block = 32
        dim_grid_x = math.ceil(width / dim_block )
        dim_grid_y = math.ceil(height / dim_block)
       
        # load CUDA code
        mod = compiler.SourceModule(open('gaussian_blur.cu').read())
        apply_filter = mod.get_function('applyFilter')
       

        # ##################
        # apply the  filter
        # ##################     
        

        for channel in (red_channel, green_channel, blue_channel):
            apply_filter(
                drv.In(channel),
                drv.Out(channel),
                np.uint32(corx),
                np.uint32(cory),
                np.uint32(width),
                np.uint32(height),
                np.uint32(width2),
                np.uint32(height2),        
                drv.In(gaussian_kernel),
                np.uint32(kernel_width),
                block=(dim_block, dim_block, 1),
                grid=(dim_grid_x, dim_grid_y)
            )     
        


        # ####################################################################### #
        # create the output array with the same shape and type as the input array #
        # ####################################################################### #
        output_array = np.empty_like(input_array)
        output_array[:, :, 0] = red_channel
        output_array[:, :, 1] = green_channel
        output_array[:, :, 2] = blue_channel

        # save result image
        Image.fromarray(output_array).save(personPath2 + '/faceBlur_{}.jpg'.format(count))       
        count = count + 1
   
    k =  cv2.waitKey(1)
    if k == 27 or count >= 300:
        break
# end time
time_ended = timeit.default_timer()

img_array = []

#For para leer imagenes desde un directorio
for x in range (0,299):
    path = '/content/images3/BlurFace/faceBlur_%01d.jpg' % x
    img = cv2.imread(path)
    img_array.append(img)

#Tamaño de la última imagen alto y ancho
height, width  = img.shape[:2]

video = cv2.VideoWriter('rosBlur.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 30, (width,height))

#For para guardar frames en un video
for i in range(len(img_array)):
    video.write(img_array[i])

video.release() 

# display total time
print('Total processing time: ', time_ended - time_started, 's')
cap.release()
cv2.destroyAllWindows()




Overwriting main.py


In [72]:
!python main.py

Total processing time:  15.852631610999197 s


In [68]:
#Librería opencv
import cv2

#Arreglo vacío
img_array = []

#For para leer imagenes desde un directorio
for x in range (0,299):
    path = '/content/images3/BlurFace/faceBlur_%01d.jpg' % x
    img = cv2.imread(path)
    img_array.append(img)

#Tamaño de la última imagen alto y ancho
height, width  = img.shape[:2]

video = cv2.VideoWriter('db1.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 30, (width,height))

#For para guardar frames en un video
for i in range(len(img_array)):
    video.write(img_array[i])

video.release()  

In [ ]:
%%writefile im.cu

#define STB_IMAGE_IMPLEMENTATION
#define STB_IMAGE_WRITE_IMPLEMENTATION
#include "stb_image.h"
#include "stb_image_write.h"
#include <iostream>
#define BLUR_SIZE 18
#define R 0
#define G 1
#define B 2
#define A 3

__global__ void blurKernel(unsigned char* in, unsigned char* out, int width, int height, int num_channel, int channel, int copy_A) {

  int col = blockIdx.x * blockDim.x + threadIdx.x+239;
  int row = blockIdx.y * blockDim.y + threadIdx.y+20;

  if(col < width+239 && row < height+20) {
    int pixVal = 0;
    int pixels = 0;
    if (copy_A)
    out[row*width*num_channel+col*num_channel+A] = in[row*width*num_channel+col*num_channel+A];
    for(int blurRow = -BLUR_SIZE; blurRow < BLUR_SIZE + 1; ++blurRow) {
      for(int blurCol = -BLUR_SIZE; blurCol < BLUR_SIZE + 1; ++blurCol) {
        int curRow = row + blurRow;
        int curCol = col + blurCol;
        if(curRow > -1 && curRow < height && curCol > -1 && curCol < width) {
          pixVal += in[curRow * width * num_channel + curCol * num_channel + channel];
          pixels++;
        }
      }
    }
    out[row * width * num_channel + col * num_channel + channel] = (unsigned char)(pixVal/pixels);
  }
}

int main() {

  int width, height,n;
  unsigned char *image = stbi_load("1366_2000.jpg",&width,&height,&n,0);
  printf("%d \n",height);
  printf("%d \n",width);

  unsigned char *output = (unsigned char*)malloc(width * height * n *sizeof(unsigned char));
  unsigned char* Dev_Input_Image = NULL;
  unsigned char* Dev_Output_Image = NULL;
  cudaMalloc((void**)&Dev_Input_Image, sizeof(unsigned char)* height * width * n);
  cudaMalloc((void**)&Dev_Output_Image, sizeof(unsigned char)* height * width * n);

  cudaMemcpy(Dev_Input_Image, image, sizeof(unsigned char) * height * width * n, cudaMemcpyHostToDevice);

  //kernel call
  dim3 blockSize(16, 16, 1);
  dim3 gridSize(width/blockSize.x, height/blockSize.y,1);
  printf("%d %d",width/blockSize.x,height/blockSize.y);
  blurKernel <<<gridSize, blockSize>>>(Dev_Input_Image, Dev_Output_Image, width, height,n,R,0);
  blurKernel <<<gridSize, blockSize>>>(Dev_Input_Image, Dev_Output_Image, width, height,n,G,0);
  blurKernel <<<gridSize, blockSize>>>(Dev_Input_Image, Dev_Output_Image, width, height,n,B,1);
  
  cudaDeviceSynchronize();

  cudaMemcpy(image, Dev_Output_Image, sizeof(unsigned char) * height * width * n, cudaMemcpyDeviceToHost);
  cudaFree(Dev_Input_Image);
  cudaFree(Dev_Output_Image);
  stbi_write_png("output_stbimage2.png", width, height, n, image, width * n);


  return 0;
} 

Overwriting im.cu


In [ ]:
!nvcc -arch=sm_37 -gencode=arch=compute_37,code=sm_37 im.cu -o him

nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
stb_image.h(4207): warning: variable "old_limit" was set but never used

stb_image.h(5121): warning: variable "idata_limit_old" was set but never used

stb_image.h(6894): warning: variable "out_size" was set but never used

stb_image.h(6895): warning: variable "delays_size" was set but never used



In [ ]:
!./him

930 
1291 
80 58